<img style="float:left;" src="https://img.icons8.com/carbon-copy/100/000000/futurama-bender.png"/>
<h1 style="text-align: center; font-size:50px;"> US election tweets </h1> 
<hr>
<hr>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objects as go

# nlp lib
import sys
import nltk
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize   
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string, collections, unicodedata
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.summarization import summarize
from gensim.summarization import keywords

# map
import seaborn as sns
import matplotlib.pyplot as plt
from geopandas.tools import geocode
!pip install geopy==1.22.0
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import math
import re
from pprint import pprint as print



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Content

The 2020 US election is happening on the 3rd November 2020 and the resulting impact to the world will no doubt be large, irrespective of which candidate is elected! After reading the two papers, here and here, I was inspired to attempt a similar sentiment analysis myself!

Columns are as follows:

- created_at: Date and time of tweet creation
- tweet_id: Unique ID of the tweet
- tweet: Full tweet text
- likes: Number of likes
- retweet_count: Number of retweets
- source: Utility used to post tweet
- user_id: User ID of tweet creator
- user_name: Username of tweet creator
- user_screen_name: Screen name of tweet creator
- user_description: Description of self by tweet creator
- user_join_date: Join date of tweet creator
- user_followers_count: Followers count on tweet creator
- user_location: Location given on tweet creator's profile
- lat: Latitude parsed from user_location
- long: Longitude parsed from user_location
- city: City parsed from user_location
- country: Country parsed from user_location
- state: State parsed from user_location
- state_code: State code parsed from user_location
- collected_at: Date and time tweet data was mined from twitter*

## Table of contents

- [Exploring the data](#a)
- [Can we detect if there are or were any attempts at manipulating the election.](#b)
- [Summary of Joe Biden tweets](#c)
- [Summary of Donald Trump tweets](#d)
- [Sentimental analysis](#e)
- [USA map](#f)

<h2 style="text-align: center;">Exploring the data</h2> <a id=a><a/>
<hr>

You will find in this section multiple graphs which give you informations about the data.

First, import the data:

In [ ]:
donald_trump = pd.read_csv('/kaggle/input/us-election-2020-tweets/hashtag_donaldtrump.csv', lineterminator='\n')
donald_trump.loc[:,'who'] = 'donald Trump'
joe_biden = pd.read_csv('/kaggle/input/us-election-2020-tweets/hashtag_joebiden.csv', lineterminator='\n')
joe_biden.loc[:,'who'] = 'joe Biden'
data = pd.concat([joe_biden,donald_trump])

In [ ]:
data.sort_values(by='created_at').info()

In [ ]:
data

Unhide cells to see the code :) 

In [ ]:
y = data.query('(who == "joe Biden") ').dropna(subset=['country']).groupby(by='country').count().tweet.sort_values(ascending=False)
x = data.query('(who == "joe Biden") ').dropna(subset=['country']).groupby(by='country').count().tweet.sort_values(ascending=False).index
y2 = data.query('(who == "donald Trump") ').dropna(subset=['country']).groupby(by='country').count().tweet.sort_values(ascending=False)
x2 = data.query('(who == "donald Trump")').dropna(subset=['country']).groupby(by='country').count().tweet.sort_values(ascending=False).index
fig = go.Figure([go.Bar(x=x, y=y, name='joe Biden'),
                 go.Bar(x=x2, y=y2, name='donald Trump')])

# Customize aspect
fig.update_layout(title_text='tweets count for each countries')
fig.update_xaxes(title='users')
fig.update_yaxes(title='tweets count')
fig.show()

In [ ]:
y = data.query('(who == "joe Biden") & (country == "United States of America")').dropna(subset=['state','country']).groupby(by='state').count().tweet.sort_values(ascending=False)
x = data.query('(who == "joe Biden") & (country == "United States of America")').dropna(subset=['state','country']).groupby(by='state').count().tweet.sort_values(ascending=False).index
y2 = data.query('(who == "donald Trump") & (country == "United States of America")').dropna(subset=['state','country']).groupby(by='state').count().tweet.sort_values(ascending=False)
x2 = data.query('(who == "donald Trump") & (country == "United States of America")').dropna(subset=['state','country']).groupby(by='state').count().tweet.sort_values(ascending=False).index
fig = go.Figure([go.Bar(x=x, y=y, name='joe Biden'),
                 go.Bar(x=x2, y=y2, name='donald Trump')])

# Customize aspect
fig.update_layout(title_text='Tweets count for each states of America')
fig.update_xaxes(title='users')
fig.update_yaxes(title='tweets count')
fig.show()

In [ ]:
y = data.query('who == "joe Biden"').groupby(by='user_name').count().tweet.sort_values(ascending=False)[:10]
x = data.query('who == "joe Biden"').groupby(by='user_name').count().tweet.sort_values(ascending=False)[:10].index
y2 = data.query('who == "donald Trump"').groupby(by='user_name').count().tweet.sort_values(ascending=False)[:10]
x2 = data.query('who == "donald Trump"').groupby(by='user_name').count().tweet.sort_values(ascending=False)[:10].index
fig = go.Figure([go.Bar(x=x, y=y, name='joe Biden'),
                 go.Bar(x=x2, y=y2, name='donald Trump')])

# Customize aspect
fig.update_layout(title_text='top 10 most active users')
fig.update_xaxes(title='users')
fig.update_yaxes(title='tweets count')
fig.show()

In [ ]:
y = data.query('who == "joe Biden"').groupby(by='source').count().tweet.sort_values(ascending=False)[:10]
x = data.query('who == "joe Biden"').groupby(by='source').count().tweet.sort_values(ascending=False)[:10].index
y2 = data.query('who == "donald Trump"').groupby(by='source').count().tweet.sort_values(ascending=False)[:10]
x2 = data.query('who == "donald Trump"').groupby(by='source').count().tweet.sort_values(ascending=False)[:10].index
fig = go.Figure([go.Bar(x=x, y=y, name='joe Biden'),
                 go.Bar(x=x2, y=y2, name='donald Trump')])

# Customize aspect
fig.update_layout(title_text='top 10 sources')
fig.update_xaxes(title='sources')
fig.update_yaxes(title='tweets count')
fig.show()

In [ ]:
y = data.query('who == "joe Biden"').sort_values('user_followers_count',ascending=False).drop_duplicates(['user_name']).user_followers_count[:10]
x = data.query('who == "joe Biden"').sort_values('user_followers_count',ascending=False).drop_duplicates(['user_name']).user_name[:10]
y2 = data.query('who == "donald Trump"').sort_values('user_followers_count',ascending=False).drop_duplicates(['user_name']).user_followers_count[:10]
x2 = data.query('who == "donald Trump"').sort_values('user_followers_count',ascending=False).drop_duplicates(['user_name']).user_name[:10]
fig = go.Figure([go.Bar(x=x, y=y, name='joe Biden'),
                 go.Bar(x=x2, y=y2, name='donald Trump')])

# Customize aspect
fig.update_layout(title_text='top 10 famous users')
fig.update_xaxes(title='users')
fig.update_yaxes(title='followers count')
fig.show()

<h2 style="text-align: center;">Can we detect if there are or were any attempts at manipulating the election. </h2><a id=b><a/>
<hr>

For this inspiration i took some assumptions. I only took into account the american top famous users and study their tweets. 

I tried to give an overview of the top famous tweets and show the similarites to Joe Biden and Donald Trump names: 

Thanks to this approach, we can show if a candidate is more criticized.

## 1 - These two functions are used to detect if the language used in the tweet is english or not. If not, we remove them.

In [ ]:
def _calculate_languages_ratios(text):
    """
    Calculate probability of given text to be written in several languages and
    return a dictionary that looks like {'french': 2, 'spanish': 4, 'english': 0}
    
    @param text: Text whose language want to be detected
    @type text: str
    
    @return: Dictionary with languages and unique stopwords seen in analyzed text
    @rtype: dict
    """

    languages_ratios = {}

    '''
    nltk.wordpunct_tokenize() splits all punctuations into separate tokens
    
    >>> wordpunct_tokenize("That's thirty minutes away. I'll be there in ten.")
    ['That', "'", 's', 'thirty', 'minutes', 'away', '.', 'I', "'", 'll', 'be', 'there', 'in', 'ten', '.']
    '''

    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens]

    # Compute per language included in nltk number of unique stopwords appearing in analyzed text
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(words)
        common_elements = words_set.intersection(stopwords_set)

        languages_ratios[language] = len(common_elements) # language "score"

    return languages_ratios

def detect_language(text):
    """
    Calculate probability of given text to be written in several languages and
    return the highest scored.
    
    It uses a stopwords based approach, counting how many unique stopwords
    are seen in analyzed text.
    
    @param text: Text whose language want to be detected
    @type text: str
    
    @return: Most scored language guessed
    @rtype: str
    """

    ratios = _calculate_languages_ratios(text)

    most_rated_language = max(ratios, key=ratios.get)

    return most_rated_language

## 2 - We apply 4 functions to get the tweets we need, clean it, display on a map the location and to give a sum up and important words on the bench of tweets.

In [ ]:
def usa_tweet(who, nb):
    """return the tweets related to who. nb is the number of tweets used"""
    
    tweets = data.query('who == "'+who+'"').sort_values('user_followers_count',ascending=False).drop_duplicates(['user_name'])[['tweet','long','lat','country']]
    tweets = tweets.dropna().loc[tweets.country=='United States of America'][:nb]
    languages = [detect_language(val) for val in tweets.tweet]
    text = tweets.loc[np.array(languages) == 'english']
    return text

def map_tweets(df):
    """return a map of the USA tweets"""
    
    m_3 = folium.Map(location=[35.32,-91.0589], tiles='cartodbpositron', zoom_start=4)

    # Add points to the map
    df = df.dropna(subset=['long','lat'])
    mc = HeatMap(data=df[['lat','long']])
    m_3.add_child(mc)

    # Display the map
    return m_3

def clean_text(df):
    """Remove HTTPS... and concatenate sentences in one string"""
    
    string = ''
    for t in df.tweet.replace('\n', ' ').str.lower():
        string = string+' '+t
    return string

def sum_up(string, ratio):
    """return sum up, the ratio determine the lenght of the summary"""
    
    print('-----sum up-----/n')
    print(summarize(string,ratio=ratio, split=True))
    print('/n-----Key words-----')
    print(keywords(string, ratio=ratio))
    

## 3 - Joe Biden tweets summary and Heat Map<a id=c><a/>

In [ ]:
df_j = usa_tweet('joe Biden', 1000)
txt_j = clean_text(df_j)
sum_up(txt_j, 0.005)

In [ ]:
map_tweets(df_j)

## 4 - Donald Trump tweets summary and Heat Map<a id=d><a/>

In [ ]:
df_d = usa_tweet('donald Trump', 1000)
txt_d = clean_text(df_d)
sum_up(txt_d, 0.005)

In [ ]:
map_tweets(df_d)

## Conclusion:

It's difficult to say if someone tries to manipulate the election. the previous summaries give us an overview of the users feeling. The two candidates has their own defects. We can add an analysis to complete the previous work.

## Let's see what are the closest words for each candidate name

In [ ]:
# We have to clean the text and split each word
stopWords = set(stopwords.words('english'))

def stop_words(sent):
    
    sent = (unicodedata.normalize('NFKD', sent)
            .encode('ascii', 'ignore')
            .decode('utf-8', 'ignore')
            .lower())
    sent = re.sub(r'[^\w\s]', '', sent)
    list_words = nltk.word_tokenize(sent)
    list_words = [w for w in list_words if w not in stopWords]
    return list_words


df_d = usa_tweet('donald Trump', 2000).tweet.str.replace('\n', ' ').str.lower()
# Split the text
all_words = [stop_words(sent) for sent in df_d]
# from gensim (It creates a neural network to trasform word into vectors)
model = Word2Vec(all_words, size=10, window=1, min_count=1, workers=4)

#### Similarities with 'joe'

In [ ]:
data_sim = model.wv.most_similar('joe',topn=25)
x = [i for i,j in data_sim]
y = [j for i,j in data_sim]

fig, ax = plt.subplots(figsize=(10,10))
sns.barplot(x=x, y=y, color="salmon", saturation=.5, ax=ax)
ax.set_title('Closest words to joe')
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
ax.set_ylim(np.min(y),1)
ax.set_xlabel('Words')
ax.set_ylabel('Proportion of similarities')

#### Similarities with 'donald'

In [ ]:
data_sim = model.wv.most_similar('donald',topn=25)

x = [i for i,j in data_sim]
y = [j for i,j in data_sim]

fig, ax = plt.subplots(figsize=(10,10))
sns.barplot(x=x, y=y, color="salmon", saturation=.5, ax=ax)
ax.set_title('Closest words to joe')
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

ax.set_ylim(np.min(y),1)
ax.set_xlabel('Words')
ax.set_ylabel('Proportion of similarities')

<h2 style="text-align: center;">Sentimental Analysis</h2> <a id=e><a/>
<hr>

For this part, i used the sentimental analyser from nltk which give us a proportion of neutrality, positivity and negativity for each sentence. It can be used only on english tweets.

>Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

Let's first take the case of Donald Trump: Over 5000 most famous tweets.

Assumptions: 
- A tweet is counted as positive if its value pos > neg
- A tweet is counted as negative if its value neg > pos
- A tweet is counted as neutral if its value neu == 1

### Donald Trump

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def analyse(who):
    sid = SentimentIntensityAnalyzer()
    neg = []
    neu = []
    pos = []
    for sentence in usa_tweet(who, 7500).tweet.str.replace('\n', ' '):
        ss = sid.polarity_scores(sentence)
        neg.append(ss[sorted(ss)[1]])
        neu.append(ss[sorted(ss)[2]])
        pos.append(ss[sorted(ss)[3]])

    result = pd.DataFrame({'pos':pos,'neg':neg,'neu':neu})

    a = (result.pos>result.neg).sum()
    b = (result.pos<result.neg).sum()
    c = (result.pos==result.neg).sum()

    
    colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
    fig = go.Figure(data=[go.Pie(labels=["positivity","negativity","neutrality"], values=[a,b,c]),])
    fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                      marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    fig.update(layout_title_text=f'Proportions of subjectivity on {who} tweets')

    fig.show()

    confidence_interval(a,b+c)

def usa_tweet(who, nb):
    """return the tweets related to who. nb is the number of tweets used"""
    
    tweets = data.query('who == "'+who+'"').sample(frac = 1).drop_duplicates(['user_name'])[['tweet','state','long','lat','country']]
    tweets = tweets.dropna().loc[tweets.country=='United States of America'][:nb]
    languages = [detect_language(val) for val in tweets.tweet]
    text = tweets.loc[np.array(languages) == 'english']
    return text

def confidence_interval(N,N2):
    """1.96 means that we are sure up to 95% that p is under low and up bounds"""
    
    p = round(N/(N+N2),3)
    low = round(p-1.96*math.sqrt(p*(1-p)/(N+N2)),3)
    up = round(p+1.96*math.sqrt(p*(1-p)/(N+N2)),3)
    print("The confidence interval for positivity tweets is : Low:{0} mean:{1} up:{2}".format(low,p,up))

In [ ]:
analyse('donald Trump')

### Joe Biden

In [ ]:
analyse('joe Biden')

We could use these previous assumptions to estimate the final result:


<h2 style="text-align: center;">USA map</h2> <a id=f><a/>
<hr>

You can find and downloads the usa shapefiles dataset to create choroplet map. Be carefull 6 states have to be removed: Guam, Puerto Rico, United States Virgin Islands, Commonwealth of the Northern Mariana Islands, American Samoa, District of Columbia. 

This map show the candidate who have a better global appreciation for each state.

In [ ]:
poly = gpd.read_file('/kaggle/input/usapolygones4/tl_2014_us_state.shp', SHAPE_RESTORE_SHX='YES')

# preprocessing
string = ''
for x in poly.NAME:
    if x not in ["Guam", "Puerto Rico", "United States Virgin Islands", "Commonwealth of the Northern Mariana Islands", "American Samoa", "District of Columbia"]:
        string = string + x + '|'
    
usa_states = data[['tweet','state','who']].dropna()
usa_states_filt = usa_states.loc[usa_states.state.str.contains(string[:-1], case=False)]
usa_states_filt = usa_states_filt.loc[~usa_states_filt.state.isin(['Baja California','Baja California Sur'])]

In [ ]:
def kfold_stratified(X, y, nb):
    """
    X: dataframe
    y: string
    nb_split: number of exemples per classes
    
    return a sample of the dataframe, same number of data for each classes
    """
    
    X_tf = pd.DataFrame(columns=X.columns)
    for name in X['who'].unique():
        for classe in X[str(y)].unique():
            X_ech = X.loc[(X[str(y)]==classe)&(X['who']==name)].sample(frac=1).iloc[:int(nb/2)]
            X_tf = pd.concat([X_tf, X_ech], axis=0, ignore_index=True)
    return X_tf

# samples
usa_states_tf = kfold_stratified(usa_states_filt, 'state',10000)

sid = SentimentIntensityAnalyzer()
    
def clean2(text):
    text = (unicodedata.normalize('NFKD', text)
            .encode('ascii', 'ignore')
            .decode('utf-8', 'ignore')
            .lower())
    text = re.sub(r'[^\w\s]', '', text)
    return text

def sent_tweets(t):
    """
    return the polarity of the sentence t
    
    1: neg
    2: neu
    3: pos
    """
    
    t = t.replace('\n', ' ')
    ss = sid.polarity_scores(clean2(t))
    if ss[sorted(ss)[1]] > ss[sorted(ss)[3]]:
        return 1
    elif ss[sorted(ss)[3]] > ss[sorted(ss)[1]]:
        return 3
    else:
        return 2
    

def sentiment(x):
    """return the % of positivity for each state"""
    df = x.value_counts()
    try:
        return (df[3]*100)/ df.sum()
    except:
        return np.nan
    
# Sentiments   
usa_states_tf['sentiment'] = usa_states_tf.tweet.apply(sent_tweets)

# Count proportions
usa_states_tf_gb = usa_states_tf.groupby(['state','who']).agg({'sentiment':sentiment})

results = pd.DataFrame(index=usa_states_tf.state.unique(), columns=['NAME'])
for data in results.index:
    if usa_states_tf_gb.loc[[(data,'donald Trump')]].values > usa_states_tf_gb.loc[[(data,'joe Biden')]].values:
        results.loc[data,'NAME'] = 1 #donald Trump
    else:
        results.loc[data,'NAME'] = 2 #joe Biden

In [ ]:
m_6 = folium.Map(location=[35.32,-81.0589], tiles='cartodbpositron', zoom_start=3)

# Add a choropleth map to the base map
Choropleth(geo_data=poly[['NAME','geometry']].set_index('NAME').__geo_interface__, 
           data=results['NAME'],
           bins=3,
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='(yellow)donald Trump vs (blue)joe Biden'
          ).add_to(m_6)

# Display the map
m_6

In [ ]:

x = usa_states_tf_gb.reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=x.state.unique(),
    y=x.loc[x.who=='donald Trump'].sentiment,
    name='donald Trump',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=x.state.unique(),
    y=x.loc[x.who=='joe Biden'].sentiment,
    name='joe Biden',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45, title='Donald Trump VS Joe Biden: Positivity proportion per states')
fig.show()

# Thanks for your time and hope this notebook will help you!!